Going to attempt to build some keras layers for text and benchmark that against datarobot text-only models on the leaderboard.   Going to use the file DR_zid_sw_2016-06-30.gz  which corresponds to https://app.datarobot.com/projects/586de94dc8089157b1c70876/models

These are the text models below

<img  src="Screen Shot 2017-01-18 at 7.27.20 AM.png"/>

### Import Data

In [1]:
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.layers.core import Dense, Dropout
from keras.models import Sequential
from keras.optimizers import Adam #https://keras.io/optimizers/
from keras.layers.normalization import BatchNormalization #https://keras.io/layers/normalization/
from keras.regularizers import WeightRegularizer #https://keras.io/regularizers/
pd.options.display.max_columns = 100

Using TensorFlow backend.


In [2]:
df = pd.read_csv('/Users/hamelhusain/Google Drive/Team Shared Folder - Zidisha/DR_zid_sw_2016-06-30.gz')

Lets tackle the text feature proposal_field first

In [3]:
df_text = df[['proposal_field', 'default_flag','part']]
#Fill null values with empty string
df.proposal_field[df.proposal_field.isnull()] = ''
df_text.head()

/Users/hamelhusain/anaconda/envs/drpy3/lib/python3.5/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


,proposal_field,default_flag,part
0,Thank you for giving me this opportunity to ap...,0,t
1,The first loan given will be used to aid in re...,1,t
2,The first loan given will be used to purchase ...,0,t
3,Thank you all the zidisha Microfinance for the...,1,t
4,"I will use KES 15,000 to add up to my savings ...",0,t


In [4]:
#set max vocabulary at 500 words
max_words = 3000
text_tokenizer = Tokenizer(nb_words=3000)

In [5]:
#train validation holdout split
train = df[df.part == 't'].proposal_field.values.tolist()
train_labels = df[df.part == 't'].default_flag.values.tolist()

validation = df[df.part == 'v'].proposal_field.values.tolist()
validation_labels =  df[df.part == 'v'].default_flag.values.tolist()

holdout = df[df.part == 'h'].proposal_field.values.tolist()

#Fit data on training set
text_tokenizer.fit_on_texts(train)

In [6]:
#Transform datasets
train_transformed = text_tokenizer.texts_to_matrix(train, mode = 'tfidf')
validation_transformed = text_tokenizer.texts_to_matrix(validation, mode = 'tfidf')

In [7]:
model = Sequential([
    BatchNormalization(input_shape=(max_words, )), #Normalizes input
    Dense(200, activation='relu', W_regularizer=WeightRegularizer(l1=0.1, l2=0.1)),
    Dropout(.1),
    Dense(50, activation='relu', W_regularizer=WeightRegularizer(l1=0.1, l2=0.1)),
    BatchNormalization(),
    Dropout(.1),
    Dense(1, activation='sigmoid')])

model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])

In [8]:
model.fit(train_transformed, train_labels, validation_data=(validation_transformed, validation_labels), 
          nb_epoch=2, batch_size=32)

Train on 20266 samples, validate on 2408 samples
Epoch 1/2
20266/20266 [==============================] - 13s - loss: 51.0529 - acc: 0.5807 - val_loss: 0.7348 - val_acc: 0.4020   64/20266 [..............................] - ETA: 44s - loss: 1391.4512 - acc: 0.5156

In [9]:
model.fit(train_transformed, train_labels, validation_data=(validation_transformed, validation_labels), 
          nb_epoch=2, batch_size=32)

Train on 20266 samples, validate on 2408 samples
Epoch 1/2
20266/20266 [==============================] - 13s - loss: 7.4694 - acc: 0.5900 - val_loss: 0.7439 - val_acc: 0.4020   64/20266 [..............................] - ETA: 14s - loss: 7.4316 - acc: 0.6250